In [15]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
import gradio as gr

# Load pre-trained models
ann_model = tf.keras.models.load_model("model_ann.h5")
cnn_model = tf.keras.models.load_model("model_cnn.h5")

# Define label mapping for EMNIST Balanced (47 classes)
# 0-9: Digits, 10-35: Uppercase A-Z, 36-46: Lowercase a-j
emnist_classes = [chr(i) for i in range(48, 58)] + [chr(i) for i in range(65, 91)] + [chr(i) for i in range(97, 107)]

# Preprocessing function
def preprocess_image(image):
    """
    Preprocess the input image to match the EMNIST format:
    - Grayscale
    - Resize to 28x28
    - Normalize pixel values to [0, 1]
    - Invert colors to match EMNIST's white-on-black format
    """
    image = image.convert("L")  # Convert to grayscale
    image = image.resize((28, 28))  # Resize to 28x28
    image = np.array(image) / 255.0  # Normalize to [0, 1]
    image = 1 - image  # Invert colors
    return image

# Prediction function
def predict_character(image, model_choice):
    # Preprocess the image
    processed_image = preprocess_image(image)
    
    if model_choice == "ANN":
        # ANN expects a flat input of shape (784,)
        input_data = processed_image.flatten().reshape(1, 784)
        prediction = ann_model.predict(input_data)
    elif model_choice == "CNN":
        # CNN expects input of shape (28, 28, 1)
        input_data = processed_image.reshape(1, 28, 28, 1)
        prediction = cnn_model.predict(input_data)
    else:
        return "Invalid Model Choice"
    
    # Get the predicted class
    predicted_class = np.argmax(prediction)
    predicted_character = emnist_classes[predicted_class]
    confidence = np.max(prediction) * 100  # Confidence as percentage
    
    return f"Prediction: {predicted_character} (Confidence: {confidence:.2f}%)"

# Gradio Interface
def interface(image, model_choice):
    return predict_character(image, model_choice)

with gr.Blocks() as gui:
    gr.Markdown("## Handwritten Character Classification")
    gr.Markdown(
        "Upload a handwritten character image (digit, uppercase letter, or lowercase letter), "
        "and select the model (ANN or CNN) for prediction."
    )
    
    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil", label="Upload Handwritten Character Image")
            model_input = gr.Radio(
                choices=["ANN", "CNN"], value="CNN", label="Select Model"
            )
        with gr.Column():
            output_label = gr.Label(label="Prediction")
    
    predict_button = gr.Button("Predict")
    predict_button.click(
        interface,
        inputs=[image_input, model_input],
        outputs=output_label
    )

# Launch the GUI
gui.launch()


* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
